# Mock AlphaGo (3) - Reinforced Learning
In this notebook, we will train the model by letting them compete each other.

In [2]:
import os, numpy as np
from caffe2.python import core, model_helper, workspace, brew, utils
from caffe2.proto import caffe2_pb2

%matplotlib inline
from matplotlib import pyplot

# how many games will be run in this tournament
# if greater than 1, make sure randomize the choice
GAMES_ITERS = 16

DEV_OPT = core.DeviceOption(caffe2_pb2.CPU, 0) # no warranty for GPU option
ROOT_FOLDER = os.path.join(os.path.expanduser('~'), 'python', 'tutorial_data','go','param') # folder stores the loss/accuracy log
DATA_FOLDER = os.path.join(os.path.expanduser('~'), 'python', 'tutorial_data','go')

### Config for black player
BLACK_WORKSPACE = os.path.join(ROOT_FOLDER,'black')
BLACK_CONV_LEVEL = 4
BLACK_FILTERS = 128
BLACK_PRE_TRAINED_ITERS = 100000
# before traning, where to load the params
BLACK_LOAD_FOLDER = os.path.join(ROOT_FOLDER, "conv={}-k={}-iter={}".format(BLACK_CONV_LEVEL,BLACK_FILTERS,BLACK_PRE_TRAINED_ITERS))
BLACK_LOAD_INIT_NET = os.path.join(BLACK_LOAD_FOLDER, "param_init_net.pb")
BLACK_LOAD_PREDICT_NET = os.path.join(BLACK_LOAD_FOLDER, "predict_net.pb")

### Config for white player
WHITE_WORKSPACE = os.path.join(ROOT_FOLDER,'white')
WHITE_CONV_LEVEL = 4
WHITE_FILTERS = 128
WHITE_PRE_TRAINED_ITERS = 100000
# before traning, where to load the params
WHITE_LOAD_FOLDER = os.path.join(ROOT_FOLDER, "conv={}-k={}-iter={}".format(WHITE_CONV_LEVEL,WHITE_FILTERS,WHITE_PRE_TRAINED_ITERS))
WHITE_LOAD_INIT_NET = os.path.join(WHITE_LOAD_FOLDER, "param_init_net.pb")
WHITE_LOAD_PREDICT_NET = os.path.join(WHITE_LOAD_FOLDER, "predict_net.pb")

# BOARD_POSITION contains SGF symbol which represents each row (or column) of the board
# It can be used to convert between 0,1,2,3... and a,b,c,d...
# Symbol [tt] or [] represents PASS in SGF, therefore is omitted
BOARD_POSITION = 'abcdefghijklmnopqrs'

print('Black {}/{}/{} vs. White {}/{}/{}'.format(
    BLACK_CONV_LEVEL, BLACK_FILTERS, BLACK_PRE_TRAINED_ITERS,
    WHITE_CONV_LEVEL, WHITE_FILTERS, WHITE_PRE_TRAINED_ITERS))

Black 4/128/100000 vs. White 4/128/100000


training params

In [52]:
BASE_LR = -0.003 # (-0.01,0) The base Learning Rate, alphago uses -0.003 and half the number every 80m steps

DEV_OPT = core.DeviceOption(caffe2_pb2.CPU, 0) # no warranty for GPU option
TRAIN_BATCHES = 16 # how many samples will be trained within one mini-batch, depends on your hardware
#PRE_TRAINED_ITERS = 60000 # [0, infinity) how many batches the model has been trained before
#SKIP_TRAINED_DATA = 0 # [0, infinity) if this is a resumed training, how many input data will be skipped
#TRAIN_ITERS = 40000 # [0, infinity) how many batches the model will be trained

# after training, where to store the params
BLACK_SAVE_FOLDER = os.path.join(ROOT_FOLDER, "RL-conv={}-k={}-iter={}".format(BLACK_CONV_LEVEL,BLACK_FILTERS,'na'))
BLACK_SAVE_INIT_NET = os.path.join(BLACK_SAVE_FOLDER, "param_init_net.pb")
BLACK_SAVE_PREDICT_NET = os.path.join(BLACK_SAVE_FOLDER, "predict_net.pb")
# after training, where to store the params
#WHITE_SAVE_FOLDER = os.path.join(ROOT_FOLDER, "conv={}-k={}-iter={}".format(WHITE_CONV_LEVEL,WHITE_FILTERS,WHITE_PRE_TRAINED_ITERS+WHITE_TRAIN_ITERS))
#WHITE_SAVE_INIT_NET = os.path.join(WHITE_SAVE_FOLDER, "param_init_net.pb")
#WHITE_SAVE_PREDICT_NET = os.path.join(WHITE_SAVE_FOLDER, "predict_net.pb")

## AlphaGo Neural Network Architecture
Refer to AlphaGo
>  We also trained a faster but less accurate rollout policy pπ(a|s), using a linear softmax of small pattern features (see Extended Data Table 4) with weights π; this achieved an accuracy of 24.2%, using just 2µs to select an action, rather than 3ms for the policy network.

### DCNN

In [4]:
def AddConvModel(model, data, conv_level=13, filters=192):
    ''' color is BLACK or WHITE. Black and white players don't share model params.
    '''
    # Layer 1: 48 x 19 x 19 -pad-> 48 x 23 x 23 -conv-> 192 x 19 x 19
    pad1 = model.PadImage(data, 'pad1', pad_t=2, pad_l=2, pad_b=2, pad_r=2, mode="constant", value=0.)
    conv1 = brew.conv(model, pad1, 'conv1', dim_in=48, dim_out=filters, kernel=5)
    relu1 = brew.relu(model, conv1, 'relu1')
    # Layer 2-12: 192 x 19 x 19 -pad-> 192 x 21 x 21 -conv-> 192 x 19 x 19
    if conv_level > 2:
        pad2 = model.PadImage(relu1, 'pad2', pad_t=1, pad_l=1, pad_b=1, pad_r=1, mode="constant", value=0.)
        conv2 = brew.conv(model, pad2, 'conv2', dim_in=filters, dim_out=filters, kernel=3)
        relu2 = brew.relu(model, conv2, 'relu2')
        relu = relu2
    #
    if conv_level > 3:
        pad3 = model.PadImage(relu2, 'pad3', pad_t=1, pad_l=1, pad_b=1, pad_r=1, mode="constant", value=0.)
        conv3 = brew.conv(model, pad3, 'conv3', dim_in=filters, dim_out=filters, kernel=3)
        relu3 = brew.relu(model, conv3, 'relu3')
        relu = relu3
    #
    if conv_level > 4:
        pad4 = model.PadImage(relu3, 'pad4', pad_t=1, pad_l=1, pad_b=1, pad_r=1, mode="constant", value=0.)
        conv4 = brew.conv(model, pad4, 'conv4', dim_in=filters, dim_out=filters, kernel=3)
        relu4 = brew.relu(model, conv4, 'relu4')
        relu = relu4
    #
    if conv_level > 5:
        pad5 = model.PadImage(relu4, 'pad5', pad_t=1, pad_l=1, pad_b=1, pad_r=1, mode="constant", value=0.)
        conv5 = brew.conv(model, pad5, 'conv5', dim_in=filters, dim_out=filters, kernel=3)
        relu5 = brew.relu(model, conv5, 'relu5')
        relu = relu5
    #
    if conv_level > 6:
        pad6 = model.PadImage(relu5, 'pad6', pad_t=1, pad_l=1, pad_b=1, pad_r=1, mode="constant", value=0.)
        conv6 = brew.conv(model, pad6, 'conv6', dim_in=filters, dim_out=filters, kernel=3)
        relu6 = brew.relu(model, conv6, 'relu6')
        relu = relu6
    #
    if conv_level > 7:
        pad7 = model.PadImage(relu6, 'pad7', pad_t=1, pad_l=1, pad_b=1, pad_r=1, mode="constant", value=0.)
        conv7 = brew.conv(model, pad7, 'conv7', dim_in=filters, dim_out=filters, kernel=3)
        relu7 = brew.relu(model, conv7, 'relu7')
        relu = relu7
    #
    if conv_level > 8:
        pad8 = model.PadImage(relu7, 'pad8', pad_t=1, pad_l=1, pad_b=1, pad_r=1, mode="constant", value=0.)
        conv8 = brew.conv(model, pad8, 'conv8', dim_in=filters, dim_out=filters, kernel=3)
        relu8 = brew.relu(model, conv8, 'relu8')
        relu = relu8
    #
    if conv_level > 9:
        pad9 = model.PadImage(relu8, 'pad9', pad_t=1, pad_l=1, pad_b=1, pad_r=1, mode="constant", value=0.)
        conv9 = brew.conv(model, pad9, 'conv9', dim_in=filters, dim_out=filters, kernel=3)
        relu9 = brew.relu(model, conv9, 'relu9')
        relu = relu9
    #
    if conv_level > 10:
        pad10 = model.PadImage(relu9, 'pad10', pad_t=1, pad_l=1, pad_b=1, pad_r=1, mode="constant", value=0.)
        conv10 = brew.conv(model, pad10, 'conv10', dim_in=filters, dim_out=filters, kernel=3)
        relu10 = brew.relu(model, conv10, 'relu10')
        relu = relu10
    #
    if conv_level > 11:
        pad11 = model.PadImage(relu10, 'pad11', pad_t=1, pad_l=1, pad_b=1, pad_r=1, mode="constant", value=0.)
        conv11 = brew.conv(model, pad11, 'conv11', dim_in=filters, dim_out=filters, kernel=3)
        relu11 = brew.relu(model, conv11, 'relu11')
        relu = relu11
    #
    if conv_level > 12:
        pad12 = model.PadImage(relu11, 'pad12', pad_t=1, pad_l=1, pad_b=1, pad_r=1, mode="constant", value=0.)
        conv12 = brew.conv(model, pad12, 'conv12', dim_in=filters, dim_out=filters, kernel=3)
        relu12 = brew.relu(model, conv12, 'relu12')
        relu = relu12
    # Layer 13: 192 x 19 x 19 -conv-> 1 x 19 x 19 -softmax-> 361
    conv13 = brew.conv(model, relu, 'conv13', dim_in=filters, dim_out=1, kernel=1)
    ## todo: bias layer?
    predict = brew.softmax(model, conv13, 'predict')
    return predict

### Accuracy

In [5]:
def AddAccuracy(model, predict, label):
    """Adds an accuracy op to the model"""
    predict_2d, _ = model.Reshape(['predict'], ['predict_2d', '_'], shape=(0,-1))
    accuracy = brew.accuracy(model, [predict_2d, label], "accuracy")
    return accuracy

### Training Operator (Backward Propagation)
The training operator is almost same as MNIST. Refer to AlphaGo
>TBD

In [6]:
def AddTrainingOperators(model, softmax, label):
    """Adds training operators to the model."""
    xent = model.LabelCrossEntropy([softmax, label], 'xent')
    # compute the expected loss
    loss = model.AveragedLoss(xent, "loss")
    # track the accuracy of the model
    AddAccuracy(model, softmax, label)
    # use the average loss we just computed to add gradient operators to the model
    model.AddGradientOperators([loss])
    # do a simple stochastic gradient descent
    ITER = brew.iter(model, "iter")
    # set the learning rate schedule
    LR = model.LearningRate(
        ITER, "LR", base_lr=BASE_LR, policy="fixed", stepsize=1, gamma=0.999 ) # when policy=fixed, stepsize and gamma are ignored
    # ONE is a constant value that is used in the gradient update. We only need
    # to create it once, so it is explicitly placed in param_init_net.
    ONE = model.param_init_net.ConstantFill([], "ONE", shape=[1], value=1.0)
    # Now, for each parameter, we do the gradient updates.
    for param in model.params:
        # Note how we get the gradient of each parameter - ModelHelper keeps
        # track of that.
        param_grad = model.param_to_grad[param]
        # The update is a simple weighted sum: param = param + param_grad * LR
        model.WeightedSum([param, ONE, param_grad, LR], param)

### Trace the training progress
Print 'accuracy' and 'loss' to file and we can monitor them elsewhere.

In [7]:
#def AddBookkeepingOperators(model):
#    """This adds a few bookkeeping operators that we can inspect later.
#    These operators do not affect the training procedure: they only collect
#    statistics and prints them to file or to logs.
#    """    
#    # Print basically prints out the content of the blob. to_file=1 routes the
#    # printed output to a file. The file is going to be stored under
#    #     root_folder/[blob name]
#    model.Print('accuracy', [], to_file=1)
#    model.Print('loss', [], to_file=1)

## Build the actual network

In [8]:
arg_scope = {"order": "NCHW"}

data = np.empty(shape=(TRAIN_BATCHES,48,19,19), dtype=np.float32)
label = np.empty(shape=(TRAIN_BATCHES,1), dtype=np.int32)

def load_net(INIT_NET, PREDICT_NET, device_opts=DEV_OPT):
    init_def = caffe2_pb2.NetDef()
    with open(INIT_NET, 'r') as f:
        init_def.ParseFromString(f.read())
        init_def.device_option.CopyFrom(device_opts)
        workspace.RunNetOnce(init_def.SerializeToString())
#    net_def = caffe2_pb2.NetDef()
#    with open(PREDICT_NET, 'r') as f:
#        net_def.ParseFromString(f.read())
#        net_def.device_option.CopyFrom(device_opts)
#        workspace.CreateNet(net_def.SerializeToString(), overwrite=True)

### Train Net Initialize
>Train Net: Blob('data','label') ==> Predict Net ==> Loss ==> Backward Propergation

In [9]:
# Initialize Black player
workspace.SwitchWorkspace(BLACK_WORKSPACE, True)
black_train_model = model_helper.ModelHelper(name="black_train_model", arg_scope=arg_scope, init_params=True)
workspace.FeedBlob("data", data)
workspace.FeedBlob("label", data)
#black_train_model.GivenTensorFill([],"data",shape=(TRAIN_BATCHES,48,19,19),values=data)
#black_train_model.GivenTensorFill([],"label",shape=(TRAIN_BATCHES,1),values=label)
predict = AddConvModel(black_train_model, "data", conv_level=BLACK_CONV_LEVEL, filters=BLACK_FILTERS)
AddTrainingOperators(black_train_model, predict, "label")
workspace.RunNetOnce(black_train_model.param_init_net)
workspace.CreateNet(black_train_model.net, overwrite=True)

# Initialize White player
workspace.SwitchWorkspace(WHITE_WORKSPACE, True)
white_train_model = model_helper.ModelHelper(name="white_train_model", arg_scope=arg_scope, init_params=True)
workspace.FeedBlob("data", data)
workspace.FeedBlob("label", data)
#white_train_model.GivenTensorFill([],"data",shape=(TRAIN_BATCHES,48,19,19),values=data)
#white_train_model.GivenTensorFill([],"label",shape=(TRAIN_BATCHES,1),values=label)
predict = AddConvModel(white_train_model, "data", conv_level=WHITE_CONV_LEVEL, filters=WHITE_FILTERS)
AddTrainingOperators(white_train_model, predict, "label")
workspace.RunNetOnce(white_train_model.param_init_net)
workspace.CreateNet(white_train_model.net, overwrite=True)

True

### Deploy Net
Build Deploy Net and Init Deploy Net with saved weight and bias.
>Predict Net: Blob('data') ==> Predict Net ==> Blob('predict')

In [10]:
# Initialize Black player
workspace.SwitchWorkspace(BLACK_WORKSPACE, True)
black_deploy_model = model_helper.ModelHelper(name="black_policy_deploy", arg_scope=arg_scope, init_params=False)
AddConvModel(black_deploy_model, "data", conv_level=BLACK_CONV_LEVEL, filters=BLACK_FILTERS)
load_net(BLACK_LOAD_INIT_NET, BLACK_LOAD_PREDICT_NET)
workspace.CreateNet(black_deploy_model.net, overwrite=True)

# Initialize White player
workspace.SwitchWorkspace(WHITE_WORKSPACE, True)
white_deploy_model = model_helper.ModelHelper(name="white_policy_deploy", arg_scope=arg_scope, init_params=False)
AddConvModel(white_deploy_model, "data", conv_level=WHITE_CONV_LEVEL, filters=WHITE_FILTERS)
load_net(WHITE_LOAD_INIT_NET, WHITE_LOAD_PREDICT_NET)
workspace.CreateNet(white_deploy_model.net, overwrite=True)

True

## Run the tournament and training

### Compete

In [55]:
from go import GameState, BLACK, WHITE, EMPTY
from preprocessing import Preprocess
from datetime import datetime

game_state = [GameState() for i in range(GAMES_ITERS)]
game_result = [0] * GAMES_ITERS # 0 - Not Ended; BLACK - Black Wins; WHITE - White Wins
p = [Preprocess()] * GAMES_ITERS
history = [[] for i in range(GAMES_ITERS)]
# board before current move
board = [p[i].state_to_tensor(game_state[i]).astype(np.float32) for i in range(GAMES_ITERS)]

# diverse the first step for GAMES_ITERS choices
workspace.SwitchWorkspace(BLACK_WORKSPACE)
workspace.FeedBlob('data', board[0])

workspace.RunNet(black_deploy_model.net)
init_move = np.reshape(workspace.FetchBlob('predict'), (-1))
init_sorted_move = np.argsort(-init_move) # shape=(361,)

current_choice = init_sorted_move[0:GAMES_ITERS]
x = current_choice/19 # tensor
y = current_choice%19 # tensor

for i in range(GAMES_ITERS):
    history[i].append(('B',x[i],y[i],board[i]))
    game_state[i].do_move(action=(x[i],y[i]),color = BLACK)
    print('game({}) step({}) black move({},{})'.format(i, 0, x[i], y[i]))

game(0) step(0) black move(3,2)
game(1) step(0) black move(3,3)
game(2) step(0) black move(16,15)
game(3) step(0) black move(2,3)
game(4) step(0) black move(2,2)
game(5) step(0) black move(16,3)
game(6) step(0) black move(3,15)
game(7) step(0) black move(16,2)
game(8) step(0) black move(4,2)
game(9) step(0) black move(5,2)
game(10) step(0) black move(13,2)
game(11) step(0) black move(8,2)
game(12) step(0) black move(12,2)
game(13) step(0) black move(6,2)
game(14) step(0) black move(11,2)
game(15) step(0) black move(10,2)


In [56]:
# for each step in the game
for step in range(1,500):

#    for i in range(GAMES_ITERS):
#        if not game_result[i]:
#            board[i] = np.append(board[i], p[i].state_to_tensor(game_state[i]).astype(np.float32), axis=0)
    board = np.concatenate([p[i].state_to_tensor(game_state[i]).astype(np.float32) for i in range(GAMES_ITERS)])

    # pass move = if all moves are illegal
    pass_move = [True] * GAMES_ITERS
    
    if step % 2 == 0:
        # black move
        workspace.SwitchWorkspace(BLACK_WORKSPACE)
        workspace.FeedBlob('data', board)
        workspace.RunNet(black_deploy_model.net)
        move = np.reshape(workspace.FetchBlob('predict'), (GAMES_ITERS,-1))
        sorted_move = np.argsort(-move)
        for i in range(GAMES_ITERS):
            if game_result[i]:
                break;
            for current_choice in sorted_move[i]:
                x = current_choice/19
                y = current_choice%19
                if game_state[i].is_legal(action=(x,y)) and not game_state[i].is_eye((x, y), BLACK):
                    pass_move[i] = False
                    history[i].append(('B',x,y,board[i]))
                    game_result[i] = game_state[i].do_move(action=(x,y),color = BLACK) # End of Game?
                    print('game({}) step({}) black move({},{})'.format(i, step, x, y))
                    break
    else:
        # white move
        workspace.SwitchWorkspace(WHITE_WORKSPACE)
        workspace.FeedBlob('data', board)
        workspace.RunNet(white_deploy_model.net)
        move = np.reshape(workspace.FetchBlob('predict'), (GAMES_ITERS,-1))
        sorted_move = np.argsort(-move)
        for i in range(GAMES_ITERS):
            if game_result[i]:
                break;
            for current_choice in sorted_move[i]:
                x = current_choice/19
                y = current_choice%19
                if game_state[i].is_legal(action=(x,y)) and not game_state[i].is_eye((x, y), WHITE):
                    pass_move[i] = False
                    history[i].append(('W',x,y,board[i]))
                    game_result[i] = game_state[i].do_move(action=(x,y),color = WHITE) # End of Game?
                    print('game({}) step({}) white move({},{})'.format(i, step, x, y))
                    break
                    
    game_result = [game_result[i] or pass_move[i] for i in range(GAMES_ITERS)]
    
    if np.all(game_result):
        break

game(0) step(1) white move(3,4)
game(1) step(1) white move(5,2)
game(2) step(1) white move(3,3)
game(3) step(1) white move(4,3)
game(4) step(1) white move(4,2)
game(5) step(1) white move(3,3)
game(6) step(1) white move(3,3)
game(7) step(1) white move(16,3)
game(8) step(1) white move(2,3)
game(9) step(1) white move(3,2)
game(10) step(1) white move(3,3)
game(11) step(1) white move(3,3)
game(12) step(1) white move(3,3)
game(13) step(1) white move(3,3)
game(14) step(1) white move(3,3)
game(15) step(1) white move(3,3)
game(0) step(2) black move(5,2)
game(1) step(2) black move(7,2)
game(2) step(2) black move(5,2)
game(3) step(2) black move(4,2)
game(4) step(2) black move(2,4)
game(5) step(2) black move(5,2)
game(6) step(2) black move(5,2)
game(7) step(2) black move(15,2)
game(8) step(2) black move(2,2)
game(9) step(2) black move(3,3)
game(10) step(2) black move(5,2)
game(11) step(2) black move(5,2)
game(12) step(2) black move(5,2)
game(13) step(2) black move(3,2)
game(14) step(2) black move(

game(0) step(17) white move(11,8)
game(1) step(17) white move(7,3)
game(2) step(17) white move(6,1)
game(3) step(17) white move(5,16)
game(4) step(17) white move(5,15)
game(5) step(17) white move(6,1)
game(6) step(17) white move(6,1)
game(7) step(17) white move(2,1)
game(8) step(17) white move(7,3)
game(9) step(17) white move(6,3)
game(10) step(17) white move(6,1)
game(11) step(17) white move(13,2)
game(12) step(17) white move(6,1)
game(13) step(17) white move(7,2)
game(14) step(17) white move(6,1)
game(15) step(17) white move(6,1)
game(0) step(18) black move(9,10)
game(1) step(18) black move(7,0)
game(2) step(18) black move(7,3)
game(3) step(18) black move(7,16)
game(4) step(18) black move(3,15)
game(5) step(18) black move(7,3)
game(6) step(18) black move(7,3)
game(7) step(18) black move(5,1)
game(8) step(18) black move(8,2)
game(9) step(18) black move(7,2)
game(10) step(18) black move(7,3)
game(11) step(18) black move(15,2)
game(12) step(18) black move(7,3)
game(13) step(18) black mo

game(0) step(33) white move(11,13)
game(1) step(33) white move(13,4)
game(2) step(33) white move(13,3)
game(3) step(33) white move(9,16)
game(4) step(33) white move(11,12)
game(5) step(33) white move(13,3)
game(6) step(33) white move(13,3)
game(7) step(33) white move(10,3)
game(8) step(33) white move(16,3)
game(9) step(33) white move(14,3)
game(10) step(33) white move(10,5)
game(11) step(33) white move(14,3)
game(12) step(33) white move(12,3)
game(13) step(33) white move(8,1)
game(14) step(33) white move(12,3)
game(15) step(33) white move(3,13)
game(0) step(34) black move(13,13)
game(1) step(34) black move(14,3)
game(2) step(34) black move(13,4)
game(3) step(34) black move(10,15)
game(4) step(34) black move(11,11)
game(5) step(34) black move(13,4)
game(6) step(34) black move(13,4)
game(7) step(34) black move(10,4)
game(8) step(34) black move(16,2)
game(9) step(34) black move(15,2)
game(10) step(34) black move(9,6)
game(11) step(34) black move(12,5)
game(12) step(34) black move(13,2)
ga

game(0) step(48) black move(2,15)
game(1) step(48) black move(3,7)
game(2) step(48) black move(18,1)
game(3) step(48) black move(17,16)
game(4) step(48) black move(12,9)
game(5) step(48) black move(14,1)
game(6) step(48) black move(18,1)
game(7) step(48) black move(16,6)
game(8) step(48) black move(2,4)
game(9) step(48) black move(18,4)
game(10) step(48) black move(17,6)
game(11) step(48) black move(10,11)
game(12) step(48) black move(16,1)
game(13) step(48) black move(15,3)
game(14) step(48) black move(9,0)
game(15) step(48) black move(5,17)
game(0) step(49) white move(2,16)
game(1) step(49) white move(3,8)
game(2) step(49) white move(18,2)
game(3) step(49) white move(17,17)
game(4) step(49) white move(13,8)
game(5) step(49) white move(13,1)
game(6) step(49) white move(18,2)
game(7) step(49) white move(16,7)
game(8) step(49) white move(3,3)
game(9) step(49) white move(16,8)
game(10) step(49) white move(16,7)
game(11) step(49) white move(13,12)
game(12) step(49) white move(18,0)
game(1

game(0) step(63) white move(3,13)
game(1) step(63) white move(9,6)
game(2) step(63) white move(10,2)
game(3) step(63) white move(18,15)
game(4) step(63) white move(13,3)
game(5) step(63) white move(9,4)
game(6) step(63) white move(10,2)
game(7) step(63) white move(18,2)
game(8) step(63) white move(2,11)
game(9) step(63) white move(16,11)
game(10) step(63) white move(3,15)
game(11) step(63) white move(8,16)
game(12) step(63) white move(10,5)
game(13) step(63) white move(16,9)
game(14) step(63) white move(16,8)
game(15) step(63) white move(6,11)
game(0) step(64) black move(2,12)
game(1) step(64) black move(9,7)
game(2) step(64) black move(10,0)
game(3) step(64) black move(10,2)
game(4) step(64) black move(10,2)
game(5) step(64) black move(8,3)
game(6) step(64) black move(10,0)
game(7) step(64) black move(16,4)
game(8) step(64) black move(1,11)
game(9) step(64) black move(15,11)
game(10) step(64) black move(17,13)
game(11) step(64) black move(7,16)
game(12) step(64) black move(9,6)
game(1

...

game(0) step(348) black move(15,12)
game(1) step(348) black move(2,16)
game(2) step(348) black move(3,17)
game(3) step(348) black move(0,15)
game(4) step(348) black move(17,12)
game(5) step(348) black move(17,6)
game(6) step(348) black move(6,10)
game(7) step(348) black move(11,1)
game(8) step(348) black move(14,1)
game(9) step(348) black move(8,1)
game(10) step(348) black move(2,7)
game(11) step(348) black move(16,11)
game(12) step(348) black move(5,13)
game(13) step(348) black move(14,14)
game(14) step(348) black move(12,0)
game(15) step(348) black move(10,1)
game(0) step(349) white move(17,16)
game(1) step(349) white move(2,17)
game(2) step(349) white move(3,18)
game(3) step(349) white move(0,18)
game(4) step(349) white move(18,11)
game(5) step(349) white move(18,6)
game(6) step(349) white move(6,11)
game(7) step(349) white move(11,0)
game(8) step(349) white move(12,0)
game(9) step(349) white move(6,0)
game(10) step(349) white move(1,6)
game(11) step(349) white move(15,11)
game(12) 

game(0) step(363) white move(3,11)
game(1) step(363) white move(2,17)
game(2) step(363) white move(8,12)
game(3) step(363) white move(14,6)
game(4) step(363) white move(6,7)
game(5) step(363) white move(5,17)
game(6) step(363) white move(1,0)
game(7) step(363) white move(18,10)
game(8) step(363) white move(14,12)
game(9) step(363) white move(18,3)
game(10) step(363) white move(18,0)
game(11) step(363) white move(14,11)
game(12) step(363) white move(1,1)
game(13) step(363) white move(13,0)
game(14) step(363) white move(4,18)
game(15) step(363) white move(18,1)
game(0) step(364) black move(3,10)
game(1) step(364) black move(0,18)
game(2) step(364) black move(8,13)
game(3) step(364) black move(15,6)
game(4) step(364) black move(6,5)
game(5) step(364) black move(1,0)
game(6) step(364) black move(1,2)
game(7) step(364) black move(18,9)
game(8) step(364) black move(15,12)
game(9) step(364) black move(17,3)
game(10) step(364) black move(16,0)
game(11) step(364) black move(12,12)
game(12) step

game(0) step(378) black move(1,18)
game(1) step(378) black move(6,15)
game(2) step(378) black move(14,7)
game(3) step(378) black move(15,17)
game(4) step(378) black move(10,0)
game(5) step(378) black move(17,13)
game(6) step(378) black move(1,2)
game(7) step(378) black move(12,18)
game(8) step(378) black move(18,14)
game(9) step(378) black move(9,0)
game(10) step(378) black move(18,9)
game(11) step(378) black move(10,4)
game(12) step(378) black move(12,18)
game(13) step(378) black move(12,0)
game(14) step(378) black move(18,13)
game(15) step(378) black move(11,7)
game(0) step(379) white move(1,17)
game(1) step(379) white move(5,16)
game(2) step(379) white move(16,7)
game(3) step(379) white move(10,17)
game(4) step(379) white move(10,1)
game(5) step(379) white move(17,14)
game(6) step(379) white move(0,1)
game(7) step(379) white move(13,18)
game(8) step(379) white move(18,11)
game(9) step(379) white move(10,0)
game(10) step(379) white move(17,1)
game(11) step(379) white move(12,12)
game

game(0) step(393) white move(16,18)
game(1) step(393) white move(1,16)
game(2) step(393) white move(8,8)
game(3) step(393) white move(10,18)
game(4) step(393) white move(13,1)
game(5) step(393) white move(4,13)
game(6) step(393) white move(8,7)
game(7) step(393) white move(14,18)
game(8) step(393) white move(15,0)
game(9) step(393) white move(18,11)
game(10) step(393) white move(18,5)
game(11) step(393) white move(15,18)
game(12) step(393) white move(18,7)
game(13) step(393) white move(2,16)
game(14) step(393) white move(18,14)
game(15) step(393) white move(6,9)
game(0) step(394) black move(17,18)
game(1) step(394) black move(0,13)
game(2) step(394) black move(8,6)
game(3) step(394) black move(5,8)
game(4) step(394) black move(14,9)
game(5) step(394) black move(4,12)
game(6) step(394) black move(12,9)
game(7) step(394) black move(17,10)
game(8) step(394) black move(13,0)
game(9) step(394) black move(17,12)
game(10) step(394) black move(18,6)
game(11) step(394) black move(18,15)
game(12

game(0) step(410) black move(4,16)
game(1) step(410) black move(17,15)
game(2) step(410) black move(12,1)
game(0) step(411) white move(3,16)
game(1) step(411) white move(18,15)
game(2) step(411) white move(10,0)
game(0) step(412) black move(14,2)
game(1) step(412) black move(17,18)
game(2) step(412) black move(16,0)
game(0) step(413) white move(15,0)
game(1) step(413) white move(18,18)
game(2) step(413) white move(16,1)
game(0) step(414) black move(14,1)
game(1) step(414) black move(17,17)
game(2) step(414) black move(5,11)
game(0) step(415) white move(13,2)
game(1) step(415) white move(18,16)
game(2) step(415) white move(17,0)
game(0) step(416) black move(14,4)
game(1) step(416) black move(16,18)
game(2) step(416) black move(0,8)
game(0) step(417) white move(13,4)
game(1) step(417) white move(16,17)
game(2) step(417) white move(13,1)
game(0) step(418) black move(15,4)
game(1) step(418) black move(11,1)
game(2) step(418) black move(5,16)
game(0) step(419) white move(16,4)
game(1) step(

### Record the game in SGF format

In [57]:
import sgf
def write_back_sgf(game_state, history, i):
    parser = sgf.Parser()
    collection = sgf.Collection(parser)
    parser.start_gametree()
    parser.start_node()
    parser.start_property('FF') # SGF format version
    parser.add_prop_value('4')
    parser.end_property()
    parser.start_property('SZ') # Board Size = 19
    parser.add_prop_value('19')
    parser.end_property()
    parser.start_property('KM') # Komi = 7.5
    parser.add_prop_value('7.5')
    parser.end_property()
    parser.start_property('PB') # Black Player = Supervised Learning / Reinforced Learning
    parser.add_prop_value('RL-{}')
    parser.end_property()
    parser.start_property('PW') # White Player = Supervised Learning / Reinforced Learning
    parser.add_prop_value('SL-{}')
    parser.end_property()
    parser.start_property('DT') # Game Date
    parser.add_prop_value(datetime.now().strftime("%Y-%m-%d"))
    parser.end_property()
    parser.start_property('RE') # Result = B+, W+, T
    winner = game_state.get_winner()
    if winner == BLACK:
        parser.add_prop_value('B+')
        winner = 'B+'
    elif winner == WHITE:
        parser.add_prop_value('W+')
        winner = 'W+'
    else:
        parser.add_prop_value('T')
        winner = 'T'
    parser.end_property()
    parser.end_node()
    
    for step in history:
        parser.start_node()
        parser.start_property(step[0]) # or W
        parser.add_prop_value(BOARD_POSITION[step[1]]+BOARD_POSITION[step[2]])
        parser.end_property()
        parser.end_node()
    
    parser.end_gametree()
    
    # record the game in SGF
    with open(os.path.join(os.path.expanduser('~'), 'python', 'tutorial_files','selfplay',
                           '({}_{}_{})vs({}_{}_{})_{}_{}_{}.sgf'.format(
                               BLACK_CONV_LEVEL, BLACK_FILTERS, BLACK_PRE_TRAINED_ITERS,
                               WHITE_CONV_LEVEL, WHITE_FILTERS, WHITE_PRE_TRAINED_ITERS,
                               winner,
                               i,
                               datetime.now().strftime("%Y-%m-%d"))), "w") as f:
        collection.output(f)

#comment out for better performance
for i in range(GAMES_ITERS):
    write_back_sgf(game_state[i], history[i], i)

## Learn from the winning games

In [58]:
iter = 0
k = 0
for i in range(GAMES_ITERS):
    winner = game_state[i].get_winner()
    for step in history[i]:
        if (step[0] == 'B' and winner == BLACK) or (step[0] == 'W' and winner == WHITE):
            data[k] = step[3]
            label[k] = step[1]*19+step[2]
            k += 1
            if k == TRAIN_BATCHES:
                iter += 1
                print('Learning {} steps in {} of {} games. {} wins'.format(iter * 32, i, GAMES_ITERS, step[0]))
                k = 0
                workspace.SwitchWorkspace(BLACK_WORKSPACE)
                workspace.FeedBlob("data", data)
                workspace.FeedBlob("label", label)
                workspace.RunNet(black_train_model.net)
                #workspace.SwitchWorkspace(WHITE_WORKSPACE)
                #workspace.FeedBlob("data", data)
                #workspace.FeedBlob("label", label)
                #workspace.RunNet(white_train_model.net)
print('Finished')

Learning 32 steps in 0 of 16 games. W wins
Learning 64 steps in 0 of 16 games. W wins
Learning 96 steps in 0 of 16 games. W wins
Learning 128 steps in 0 of 16 games. W wins
Learning 160 steps in 0 of 16 games. W wins
Learning 192 steps in 0 of 16 games. W wins
Learning 224 steps in 0 of 16 games. W wins
Learning 256 steps in 0 of 16 games. W wins
Learning 288 steps in 0 of 16 games. W wins
Learning 320 steps in 0 of 16 games. W wins
Learning 352 steps in 0 of 16 games. W wins
Learning 384 steps in 0 of 16 games. W wins
Learning 416 steps in 0 of 16 games. W wins
Learning 448 steps in 1 of 16 games. B wins
Learning 480 steps in 1 of 16 games. B wins
Learning 512 steps in 1 of 16 games. B wins
Learning 544 steps in 1 of 16 games. B wins
Learning 576 steps in 1 of 16 games. B wins
Learning 608 steps in 1 of 16 games. B wins
Learning 640 steps in 1 of 16 games. B wins
Learning 672 steps in 1 of 16 games. B wins
Learning 704 steps in 1 of 16 games. B wins
Learning 736 steps in 1 of 16 games

Learning 5856 steps in 14 of 16 games. W wins
Learning 5888 steps in 14 of 16 games. W wins
Learning 5920 steps in 14 of 16 games. W wins
Learning 5952 steps in 14 of 16 games. W wins
Learning 5984 steps in 14 of 16 games. W wins
Learning 6016 steps in 14 of 16 games. W wins
Learning 6048 steps in 14 of 16 games. W wins
Learning 6080 steps in 14 of 16 games. W wins
Learning 6112 steps in 14 of 16 games. W wins
Learning 6144 steps in 14 of 16 games. W wins
Learning 6176 steps in 15 of 16 games. W wins
Learning 6208 steps in 15 of 16 games. W wins
Learning 6240 steps in 15 of 16 games. W wins
Learning 6272 steps in 15 of 16 games. W wins
Learning 6304 steps in 15 of 16 games. W wins
Learning 6336 steps in 15 of 16 games. W wins
Learning 6368 steps in 15 of 16 games. W wins
Learning 6400 steps in 15 of 16 games. W wins
Learning 6432 steps in 15 of 16 games. W wins
Learning 6464 steps in 15 of 16 games. W wins
Learning 6496 steps in 15 of 16 games. W wins
Learning 6528 steps in 15 of 16 ga

### Save the RL model

In [59]:
def save_net(INIT_NET, PREDICT_NET, model) :
    with open(PREDICT_NET, 'wb') as f:
        f.write(model.net._net.SerializeToString())
    init_net = caffe2_pb2.NetDef()
    for param in model.params:
        blob = workspace.FetchBlob(param)
        shape = blob.shape
        op = core.CreateOperator("GivenTensorFill", [], [param],arg=[ utils.MakeArgument("shape", shape),utils.MakeArgument("values", blob)])
        init_net.op.extend([op])
    init_net.op.extend([core.CreateOperator("ConstantFill", [], ["data"], shape=(1,30,30))])
    with open(INIT_NET, 'wb') as f:
        f.write(init_net.SerializeToString())
        
if not os.path.exists(BLACK_SAVE_FOLDER):
    os.makedirs(BLACK_SAVE_FOLDER)
    save_net(BLACK_SAVE_INIT_NET, BLACK_SAVE_PREDICT_NET, black_deploy_model)
    print('Params saved to {}'.format(BLACK_SAVE_FOLDER))